In [65]:
from imports import *
# sklearn
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans as sklearn_kmeans
from sklearn import metrics
from  sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.ensemble import BaggingClassifier
from sklearn.grid_search import GridSearchCV

from datetime import datetime
%matplotlib inline
#sqlContext = SQLContext(sc)


### Get Data

In [3]:
with open("./data/values.csv",'rb') as f:
    values = np.array([ [float(item) for item in line.strip().split(',')] for line in f ])
print values.shape
with open('./data/labels.csv','rb') as f:
    labels = np.ravel( np.array([ line.strip().split(',') for line in f ]) )
print labels.shape
with open('./data/test-values.csv','rb') as f:
    test_values = np.array([ [float(item) for item in line.strip().split(',')] for line in f ])
print test_values.shape

(60000, 784)
(60000,)
(10000, 784)


# submission file
<pre>
# comment
# comment
team name, make this consistent across all submissions. This will appear in the leader board.>
timestamp of sumbission in format 2015-12-xxT00:00:00
submission name-anything you want. This will appear in the leader board.
# data is 1 integer per row. there should be 10000 of them.
</pre>

In [41]:
def create_submission_file(submission_name,comment,fileName,pred):
    team_name = 'blehmanade'
    timestamp = datetime.now()
    header = '#{}\n{}\n{}\n{}\n'.format(comment,team_name,timestamp,submission_name)
    with open(fileName,'wb') as submission:
        submission.write(header)
        for prediction in predictions:
            submission.write(prediction+'\n')

### Method 1

In [ ]:
nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(values, labels)

In [ ]:
with open('data/KNeighborsClassifier_ball_tree_n10.pkl','wb') as f:
    pickle.dump(nbrs,f)

In [ ]:
predictions = nbrs.predict(test_values)
with open('data/KNeighborsClassifier_ball_tree_n10_predictions.pkl','wb') as f:
    pickle.dump(predictions,f)
predictions.shape

In [ ]:
comment = "KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(values, labels)"
fileName = 'KNeighborsClassifier_ball_tree_n10_predictions.submission'
submission_name = 'love_thy_neighbor'
create_submission_file(submission_name,comment,fileName,predictions)

In [ ]:
with open('data/KNeighborsClassifier_ball_tree_n10.pkl','rb') as f:
    nbrs = pickle.load(f)
nbrs.score(values, labels) # output: 0.97499999999999998

### Method 2

In [ ]:
nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree').fit(values, labels)

In [ ]:
predictions = nbrs.predict(test_values)
with open('data/KNeighborsClassifier_kd_tree_n10_predictions.pkl','wb') as f:
    pickle.dump(predictions,f)
predictions.shape

In [ ]:
comment = "KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree').fit(values, labels)"
fileName = 'KNeighborsClassifier_kd_tree_n10_predictions.submission'
submission_name = 'kid_neighbors_tree_fort'
create_submission_file(submission_name,comment,fileName,predictions)
!tail KNeighborsClassifier_kd_tree_n10_predictions.submission

### Method 3

In [ ]:
nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='brute').fit(values, labels)

In [ ]:
predictions = nbrs.predict(test_values)
fileName = 'KNeighborsClassifier_brute_n10_predictions'
with open('data/'+fileName+'.pkl','wb') as f:
    pickle.dump(predictions,f)
predictions.shape

In [ ]:
comment = "KNeighborsClassifier(n_neighbors=10, algorithm='kd_tree').fit(values, labels)"
submission_name = 'brutal_neighbors'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)
!tail KNeighborsClassifier_brute_n10_predictions.submission

### Method 4

In [ ]:
ada = AdaBoostClassifier().fit(values,labels)

In [ ]:
with open('data/AdaBoostClassifier.pkl','wb') as f:
    pickle.dump(ada,f)

In [ ]:
predictions = ada.predict(test_values)
fileName = 'adaBoost_SGDClassifier_predictions'
with open('data/'+fileName+'.pkl','wb') as f:
    pickle.dump(predictions,f)
predictions.shape

In [ ]:
comment = "AdaBoostClassifier().fit(values,labels)"
submission_name = 'adaboy_charlie_brown'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)
!head adaBoost_DecisionTreeClassifier_predictions.submission

In [ ]:
ada.score(values,labels) #output: 0.7265166666666667

### Method 5

In [ ]:
ada = AdaBoostClassifier(base_estimator=SGDClassifier(),algorithm='SAMME').fit(values,labels)


In [ ]:
print ada.score(values,labels)

In [ ]:
predictions = ada.predict(test_values)
fileName = 'adaBoost_SGDClassifier_predictions'
with open('data/'+fileName+'.pkl','wb') as f:
    pickle.dump(predictions,f)
predictions.shape

In [ ]:
comment = "AdaBoostClassifier(base_estimator=SGDClassifier(),algorithm='SAMME').fit(values,labels)"
submission_name = 'ada_unicorn'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)
!head adaBoost_SGDClassifier_predictions.submission

### Method 6
SVD (n_components=20) to reduce the dimensions in the dataset for KNN.
<img src="./img/method6.png" alt="Drawing" style="width: 250px;"/>

In [ ]:
#from sklearn.decomposition import TruncatedSVD
explained_variance = []
svd_component_range = range(0,91,5)
for i in svd_component_range:
    if i == 0:
        i = 1
    svd = TruncatedSVD(n_components=i, random_state=42)
    X_svd = svd.fit_transform(values)
    explained_variance.append(svd.explained_variance_ratio_.sum())
    print i

expVar = pd.DataFrame({'components':svd_component_range
                       ,'explained_variance':explained_variance
                      })
display(expVar)
display(expVar.plot(x='components',y='explained_variance'))

In [ ]:
svd = TruncatedSVD(n_components=20, random_state=42)
X_svd = svd.fit_transform(values)
nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(X_svd, labels)
#predictions = nbrs.predict(test_values)
score = nbrs.score(X_svd, labels)
print score

In [ ]:
display(type(test_values))
display(test_values.shape)
display(test_values[:10])

In [ ]:
X = svd.transform(test_values)
predictions = nbrs.predict(X)

In [ ]:
fileName = 'KNeighborsClassifier_ball_tree_n10_SVD20'
with open('data/'+fileName+'.pkl','wb') as f:
    pickle.dump(predictions,f)
predictions.shape

In [ ]:
comment = "KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(X_svd, labels)"
submission_name = 'unicorn_next_door'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)
!head KNeighborsClassifier_ball_tree_n10_SVD20.submission

### Method 6b
SVD (n_components=90) to reduce the dimensions in the dataset for KNN.

In [ ]:
svd = TruncatedSVD(n_components=90, random_state=42)
X_svd = svd.fit_transform(values)
nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(X_svd, labels)
predictions = nbrs.predict(svd.transform(test_values))
#score = nbrs.score(X_svd, labels)
#print score
fileName = 'KNeighborsClassifier_ball_tree_n10_SVD90'
with open('data/'+fileName+'.pkl','wb') as f:
    pickle.dump(predictions,f)
print predictions.shape
submission_name = 'unicorn_next_door_more'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)
!head KNeighborsClassifier_ball_tree_n10_SVD90.submission

In [ ]:
score = nbrs.score(X_svd, labels)
print score # output: 0.9792

### Method 6c 
Playing with the number of components from 6b.  

note: `labels = np.ravel(labels)`

In [ ]:
svd_component_range = range(89,120)
scores = []
for i in svd_component_range:
    if i == 90:
        continue
    svd = TruncatedSVD(n_components=i, random_state=42)
    X_svd = svd.fit_transform(values)
    nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(X_svd, labels)
    score = nbrs.score(X_svd, labels)
    scores.append(score)
    print "{}: {}".format(i,score)
    #score = nbrs.score(X_svd, labels)
    #print score
    predictions = nbrs.predict(svd.transform(test_values))
    fileName = 'KNeighborsClassifier_ball_tree_n10_SVD'+str(i)
    with open('data/'+fileName+'.pkl','wb') as f:
        pickle.dump(predictions,f)
    submission_name = 'mo_unicorn'+str(i)
    create_submission_file(submission_name,comment,fileName+'.submission',predictions)
scores_df = pd.DataFrame({'components': svd_component_range
                       ,'scores':scores
                      })
display(scores_df)
display(scores_df.plot(x='components',y='scores'))

#89: 0.97935
#91: 0.979133333333
#92: 0.9791
#93: 0.9791
#94: 0.97925
#95: 0.97905

In [ ]:
svd_component_range = range(90,58,-2)
svd_component_range.remove(90)
scores = []
for i in svd_component_range:
    svd = TruncatedSVD(n_components=i, random_state=42)
    X_svd = svd.fit_transform(values)
    nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(X_svd, labels)
    score = nbrs.score(X_svd, labels)
    scores.append(score)
    print "{}: {}".format(i,score)
    #score = nbrs.score(X_svd, labels)
    #print score
    predictions = nbrs.predict(svd.transform(test_values))
    fileName = 'KNeighborsClassifier_ball_tree_n10_SVD'+str(i)
    with open('data/'+fileName+'.pkl','wb') as f:
        pickle.dump(predictions,f)
    submission_name = 'mo_unicorn'+str(i)
    create_submission_file(submission_name,comment,fileName+'.submission',predictions)
scores_df = pd.DataFrame({'components': svd_component_range
                       ,'scores':scores
                      })
display(scores_df)
display(scores_df.plot(x='components',y='scores'))



<img src="./img/method6b2.png" alt="Drawing" style="width: 250px;"/>


components	scores  
0	58	0.980583  
1	56	0.980483  
2	54	0.980800  
3	52	0.980683  
4	50	0.980850  
5	48	0.980783  
6	46	0.980633  
7	44	0.980583  
8	42	0.980417  
9	40	0.980567  
10	38	0.980400  
11	36	0.980467  
12	34	0.980250  
13	32	0.980050  
14	30	0.980000  
15	28	0.979100  
16	26	0.978683  
17	24	0.978000  
18	22	0.977317  
19	20	0.974817  
20	18	0.972900  
21	16	0.970883  
<img src="./img/method6b.png" alt="Drawing" style="width: 250px;"/>


In [ ]:
svd_component_range = range(60,14,-2)
svd_component_range.remove(60)
scores = []
for i in svd_component_range:
    svd = TruncatedSVD(n_components=i, random_state=42)
    X_svd = svd.fit_transform(values)
    nbrs = KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree').fit(X_svd, labels)
    score = nbrs.score(X_svd, labels)
    scores.append(score)
    print "{}: {}".format(i,score)
    #score = nbrs.score(X_svd, labels)
    #print score
    predictions = nbrs.predict(svd.transform(test_values))
    fileName = 'KNeighborsClassifier_ball_tree_n10_SVD'+str(i)
    with open('data/'+fileName+'.pkl','wb') as f:
        pickle.dump(predictions,f)
    submission_name = 'mo_unicorn'+str(i)
    create_submission_file(submission_name,comment,fileName+'.submission',predictions)
scores_df = pd.DataFrame({'components': svd_component_range
                       ,'scores':scores
                      })
display(scores_df)
display(scores_df.plot(x='components',y='scores'))


### Method 7
Consider using PCA with Kmeans

In [ ]:
pca_component_range = range(0,81,5)
explained_variance_list = []
for i in pca_component_range:
    if i == 0:
        i=1
    X_pca = PCA(n_components=i).fit(values)
    explained_var = X_pca.explained_variance_ratio_.sum()
    print "{}: {}".format(i, explained_var)
    explained_variance_list.append(explained_var)
scores_df = pd.DataFrame({'components': pca_component_range
                       ,'explained_variance':explained_variance_list
                      })
display(scores_df)
display(scores_df.plot(x='components',y='explained_variance'))

In [ ]:
pca = PCA(n_components=80).fit(values)
X_pca = pca.transform(values)

In [ ]:
km = sklearn_kmeans(n_clusters=10).fit(X_pca,labels)

In [ ]:
predictions = km.predict(pca.transform(test_values))

In [ ]:
predictions[:10]

In [ ]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_)
           ))

# k-means++   91.27s    152992542352   0.485   0.496   0.490   0.361   0.485
#bench_k_means(sklearn_kmeans(init='k-means++', n_clusters=10, n_init=10), name="k-means++", data=values)

# PCA-based   31.59s    114256441871   0.765   0.418   0.541   0.196   0.418
bench_k_means(sklearn_kmeans(init=pca.components_, n_clusters=10, n_init=1), name="PCA-based", data=values)

### Method 8
Consider using `sklear.preprocessing.scale` to scale the data from the mean, then apply PCA and Kmeans

In [ ]:
pca.components_

### Method 9
[linear-support-vector-machines](http://www.scipy-lectures.org/packages/scikit-learn/#linear-support-vector-machines)

In [ ]:
from sklearn import svm
svc = svm.SVC(kernel='linear')

In [ ]:
svc.fit(X_pca, labels) 

In [ ]:
svc.score(values, labels) 

In [ ]:
svc = svm.SVC(kernel='poly', degree=3)
svc = svm.SVC(kernel='rbf')

### Method 10
GradientBoostingClassifier

In [6]:
gboost = GradientBoostingClassifier.fit_transform(values,labels)

TypeError: unbound method fit_transform() must be called with GradientBoostingClassifier instance as first argument (got ndarray instance instead)

### Method 11 
NearestCentroid

In [33]:
predictions = {}
for shrinkage in [None,0.1, 0.5,1.5,2.5]: 
    nc = NearestCentroid(shrink_threshold=shrinkage).fit(values, labels)
    predictions[str(shrinkage)] = nc.predict(test_values)
    

In [38]:
vals = [ a-b for a,b in zip([int(i) for i in predictions['None']], [int(i) for i in predictions['0.1']]) ]

In [39]:
for item in vals:
    if item !=0:
        print item

5
2
2
9
1
5
5
5


In [43]:
for prediction in predictions:
    fileName = 'NearestCentroid_shrinkage_'+str(prediction)
    with open('data/'+fileName+'.pkl','wb') as f:
        pickle.dump(predictions[prediction],f)
    comment = 'NearestCentroid(shrink_threshold=shrinkage).fit(values, labels)'
    submission_name = 'ity_bity_'+prediction
    create_submission_file(submission_name,comment,fileName+'.submission',predictions[prediction])

In [49]:
nc = NearestCentroid(shrink_threshold=2.5).fit(values, labels)
nc.score(values,labels)

0.80774999999999997

### Method 12 
BaggingClassifier

In [53]:
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)
bg = bagging.fit(values,labels)
bg.score(values,labels)

0.97511666666666663

In [54]:
predictions = bg.predict(test_values)

In [57]:
predictions[:10]

array(['7', '2', '1', '0', '4', '1', '4', '9', '5', '9'], 
      dtype='|S1')

In [58]:
with open('BaggingClassifier_KNeighborsClassifier_max_samples0.5_max_features0.5'+'.pkl','wb') as bgFile:
    pickle.dump(bg,bgFile)
with open('BaggingClassifier_KNeighborsClassifier_max_samples0.5_max_features0.5_predictions'+'.pkl','wb') as bgFile:
    pickle.dump(predictions,bgFile)


In [59]:
fileName='BaggingClassifier_KNeighborsClassifier_max_samples0.5_max_features0.5'
comment = 'BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)'
submission_name = 'we_gotta_first_timer_here'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)

In [60]:
svd = TruncatedSVD(n_components=50, random_state=42)
X_svd = svd.fit_transform(values)
bagging = BaggingClassifier(KNeighborsClassifier(n_neighbors=10))
bg = bagging.fit(X_svd,labels)
print bg.score(X_svd,labels)
predictions = bg.predict(svd.transform(test_values))
fileName='BaggingClassifier_KNeighborsClassifier_n_neighbors10_svd_'
with open(fileName+'model'+'.pkl','wb') as bgFile:
    pickle.dump(bg,bgFile)
with open(fileName+'predictions'+'.pkl','wb') as bgFile:
    pickle.dump(predictions,bgFile)
comment = 'BaggingClassifier(KNeighborsClassifier(n_neighbors=10))'
submission_name = 'movie_night:  http://bit.ly/pooty'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)

0.980583333333


In [62]:
X_svd.shape

(60000, 50)

In [63]:
svd = TruncatedSVD(n_components=100, random_state=42)
X_svd = svd.fit_transform(values)
bagging = BaggingClassifier(KNeighborsClassifier(n_neighbors=10),max_features=0.5)
bg = bagging.fit(X_svd,labels)
print bg.score(X_svd,labels)
predictions = bg.predict(svd.transform(test_values))
fileName='BaggingClassifier_KNeighborsClassifier_n_neighbors10_svd_'
with open(fileName+'model'+'.pkl','wb') as bgFile:
    pickle.dump(bg,bgFile)
with open(fileName+'predictions'+'.pkl','wb') as bgFile:
    pickle.dump(predictions,bgFile)
comment = 'BaggingClassifier(KNeighborsClassifier(n_neighbors=10))'
submission_name = 'movie_night:  http://bit.ly/pooty'
create_submission_file(submission_name,comment,fileName+'.submission',predictions)

0.97925


In [ ]:
param_grid = {'algorithm' : ['ball_tree', 'kd_tree']
              ,'leaf_size' : [10,30,50,100]
              ,'metric': ['euclidean','manhattan','chebyshev','minkowski','wminkowski','seuclidean','mahalanobis','hamming','canberra','braycurtis']
              ,'n_neighbors':[1,5,11,21,31]
             }
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid)
grid_search.fit(values,labels)

In [ ]:
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [ ]:
report(grid_search.grid_scores_)

In [ ]:
with open('grid_search.pkl','wb') as f:
    pickle.dump(grid_search,f)